# Plot and Analyze the Closest Team to each player

# Imports

In [1]:
import pandas as pd
from functions import *
%run functions.py
import folium 

In /Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be re

## Data

In [2]:
player_df = pd.read_csv("data/all_players_full_info.csv", index_col=0)
arena_df = pd.read_csv("data/arena_df.csv")
arena_df.columns = ['Team_name','Arena_name', 'Coordinates', 'Canadian_team']

### Adding Coordinates to each df

In [3]:
player_df['Lat'] = [x.split(',')[0] for x in player_df.coordinates]
player_df['Lng'] = [x.split(',')[1] for x in player_df.coordinates]

In [4]:
arena_df['Lat'] = [x.split(',')[0] for x in arena_df.Coordinates]
arena_df['Lng'] = [x.split(',')[1] for x in arena_df.Coordinates]

# EDA

## Count of players to teams in each country

In [5]:
# which country has the most players per team in country?
can_players = player_df[player_df['birthCountry'] == 'CAN']
us_players = player_df[player_df['birthCountry'] == 'USA']
count_of_can_teams = int(arena_df['Canadian_team'].sum())
count_of_amer_teams = arena_df.shape[0] - count_of_can_teams

In [6]:
# Percentage of canadians per canadian teams = ~51
can_players.shape[0] / count_of_can_teams

50.857142857142854

In [7]:
# Percentage of americans per american team = ~9
us_players.shape[0] / count_of_amer_teams

9.416666666666666

## How many players per province/state?

In [8]:
players_by_prov_state = player_df.groupby('birthStateProvince').count()['id']
# Canadian provinces make up the top 4 values
players_by_prov_state.sort_values(ascending=False)[0:5]

birthStateProvince
ON    164
QC     48
AB     43
BC     41
MN     34
Name: id, dtype: int64

In [12]:
# 42 different states or provinces represented! 
players_by_prov_state.count()

42

# Plotting Birthplaces

In [13]:
# using this function to help make the coordinates I receive into strings
str_coords = turn_coords_to_correct_format("47.1428° N, 99.7812° W")
str_coords

'47.1428,-99.7812'

In [16]:
# center of us and canada apprently is in north dakota (Robinson, N.D)
# according to https://www.nytimes.com/2017/01/25/science/north-america-geographical-center-north-dakota.html
# this is a little too high, so I'll make it lower
center_coords_str = '47.1428,-99.7812'
center_lat = float(center_coords_str[0:7]) - 4
center_lng = float(center_coords_str[-8:]) 

In [17]:
my_map = folium.Map(location=[center_lat, center_lng], zoom_start=4)

In [18]:
# adding players
for idx in range(len(player_df)):
    player_name = "<i>" + player_df['team_name'][idx] + "</i>"
    folium.CircleMarker(
    [player_df['Lat'][idx], player_df['Lng'][idx]], 
        radius=2, line_color='black', fill_color='gray', fill_opacity=0.1,
        popup=player_name, tooltip=player_df['fullName'][idx]
    ).add_to(my_map)
# adding arenas
for idx in range(len(arena_df)):
    arena_name = "<i>" + arena_df['Arena_name'][idx] + "</i>"
    folium.CircleMarker(
    [arena_df['Lat'][idx], arena_df['Lng'][idx]], 
        radius=15, line_color='red', fill_color='red', fill_opacity=0.4,
        popup=arena_name, tooltip=arena_df['Team_name'][idx]
    ).add_to(my_map)

In [19]:
my_map

# Saving Data

In [22]:
my_map.save("data/player_and_arena_locations.html")

In [23]:
player_df.to_csv("data/all_players_w_locations.csv")
arena_df.to_csv("data/arenas_w_locations.csv")